### Import the needed packages

In [ ]:
# package for using the the custom scripts 
import sys
sys.path.insert(0, "./../")
# package for handling files and directories
import os

import wandb
# a custom package for handling the data
import utilities as utils

from huggingface_hub import HfApi

from dotenv import load_dotenv

load_dotenv()

### Global variable

In [ ]:
path_full = os.environ.get("ORI_PATH")
path_dataset_ori = os.path.join(path_full, "datasets", "splitted", "original")
path_dataset_aug = os.path.join(path_full, "datasets", "splitted", "augmented")
path_model = os.path.join(path_full, "data", "model")
wandb_init_name = "implement weight training"
hf_commit_message = "implement weight training"
hyperparam = {
    "batch_size": 16,
    "img_size": 128,
    "shuffle": True,
    "model_type": "U-net",
    "metrics": ["accuracy", "MeanIoU", "Precision", "Recall"],
    "split_ratio": [7,1,2],
    "dataset_balance": True,
    "weighted_training": True,
}
ori_path = {
    "train": os.path.join(path_dataset_ori, "train"),
    "val": os.path.join(path_dataset_ori, "val"),
    "test": os.path.join(path_dataset_ori, "test"),
}
aug_path = {
    "train": os.path.join(path_dataset_aug, "train"),
    "val": os.path.join(path_dataset_aug, "val"),
    "test": os.path.join(path_dataset_aug, "test"),
}

### Import the images and mask

In [ ]:
# get the original dataset
train_ori_img, train_ori_mask = utils.load_img_mask(ori_path["train"])
val_ori_img, val_ori_mask = utils.load_img_mask(ori_path["val"])
test_ori_img, test_ori_mask = utils.load_img_mask(ori_path["test"])

# get the augmented dataset
train_aug_img, train_aug_mask = utils.load_img_mask(aug_path["train"])
val_aug_img, val_aug_mask = utils.load_img_mask(aug_path["val"])
test_aug_img, test_aug_mask = utils.load_img_mask(aug_path["test"])

### Create the batched dataset

In [ ]:
# create the tensor dataset for original dataset
trainset_ori = utils.create_dataset(train_ori_img, train_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])
valset_ori = utils.create_dataset(val_ori_img, val_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])
testset_ori = utils.create_dataset(test_ori_img, test_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])

# create the tensor dataset for augmented dataset
trainset_aug = utils.create_dataset(train_aug_img, train_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])
valset_aug = utils.create_dataset(val_aug_img, val_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])
testset_aug = utils.create_dataset(test_aug_img, test_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])

### Calculate label weight

### Train model

In [ ]:
# create the unet model
unet_custom = utils.custom_unet()
file_model_name = "unet_model_ori"
# train the model using original dataset
unet_custom, history, ev_result = utils.train_model(unet_custom, trainset_ori, valset_ori, testset_ori, path_model, file_model_name)
print(f"test loss: {ev_result[0]}", f"test mean_px_acc: {ev_result[1]}",
        f"test auc: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

In [6]:
api = HfApi()
repo = api.create_repo("CAD-Glaucoma_Segmentation", repo_type="model", exist_ok=True)

In [7]:
hyperparam["aug"] = False
wandb.init(project="CAD - Glaucoma Segmentation",
            name=f"ori - {wandb_init_name}",
            config=hyperparam)
for epoch in range(len(history.history["loss"])):
    wandb.log({
        "train_loss": history.history["loss"][epoch],
        "train_accuracy": history.history["mean_px_acc"][epoch],
        "train_auc": history.history["auc"][epoch],
        "train_precision": history.history["precision"][epoch],
        "train_recall": history.history["recall"][epoch],
        "val_loss": history.history["val_loss"][epoch],
        "val_accuracy": history.history["val_mean_px_acc"][epoch],
        "val_auc": history.history["val_auc"][epoch],
        "val_precision": history.history["val_precision"][epoch],
        "val_recall": history.history["val_recall"][epoch],
    })
wandb.log_artifact(os.path.join(path_model, f"{file_model_name}.h5"))
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bugi-sulistiyo (bugi-sulistiyo-universitas-mulawarman). Use `wandb login --relogin` to force relogin


train_accuracy,▁▂▄▆▇▇████
train_auc,▁▄▆▆▇▇▇███
train_loss,█▅▄▄▃▃▂▂▂▁
train_precision,▁▄▅▆▇▇████
train_recall,▁▃▃▄▅▆▇███
val_accuracy,▁▇████████
val_auc,██▂▁▂▁▁▂▂▁
val_loss,▁▁▃▄▃▅▅▆▆█
val_precision,▁▁▁█▇▃▃▃▃▃
val_recall,▁▁▁▁▂█████
train_accuracy,0.97397


In [8]:
api.upload_file(
    repo_id=repo.repo_id,
    path_or_fileobj=os.path.join(path_model, f"{file_model_name}.h5"),
    path_in_repo=f"model/{file_model_name}.h5",
    repo_type=repo.repo_type,
    commit_message=hf_commit_message,
)

unet_model_ori.h5: 100%|██████████| 1.57M/1.57M [00:01<00:00, 814kB/s] 


CommitInfo(commit_url='https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation/commit/c3256adbd32033952cce414f3e0f6f4e554b56af', commit_message='implement weight training', commit_description='', oid='c3256adbd32033952cce414f3e0f6f4e554b56af', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation', endpoint='https://huggingface.co', repo_type='model', repo_id='bugi-sulistiyo/CAD-Glaucoma_Segmentation'), pr_revision=None, pr_num=None)

In [9]:
# create the unet model
unet_custom = utils.custom_unet()
file_model_name = "unet_model_aug"
# train the model using augmented dataset
unet_custom, history, ev_result = utils.train_model(unet_custom, trainset_aug, valset_aug, testset_aug, path_model, file_model_name)
print(f"test loss: {ev_result[0]}", f"test mean_px_acc: {ev_result[1]}",
        f"test auc: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

Epoch 1/10
16/16 [==============================] - 22s 482ms/step - loss: 0.0080 - mean_px_acc: 0.5325 - auc: 0.8262 - precision: 0.6938 - recall: 0.5863 - val_loss: 0.0095 - val_mean_px_acc: 0.4410 - val_auc: 0.7837 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
16/16 [==============================] - 4s 90ms/step - loss: 0.0057 - mean_px_acc: 0.6357 - auc: 0.9106 - precision: 0.8086 - recall: 0.6938 - val_loss: 0.0092 - val_mean_px_acc: 0.9528 - val_auc: 0.9104 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
16/16 [==============================] - 3s 94ms/step - loss: 0.0052 - mean_px_acc: 0.7624 - auc: 0.9280 - precision: 0.8311 - recall: 0.7060 - val_loss: 0.0104 - val_mean_px_acc: 0.9841 - val_auc: 0.6378 - val_precision: 1.0000 - val_recall: 3.1430e-05
Epoch 4/10
16/16 [==============================] - 4s 96ms/step - loss: 0.0048 - mean_px_acc: 0.8513 - auc: 0.9414 - precision: 0.8566 - recall: 0.7330 - val_loss: 0.0103 - val_mean_px_acc: 0.98

In [10]:
hyperparam["aug"] = True
wandb.init(project="CAD - Glaucoma Segmentation",
            name=f"aug - {wandb_init_name}",
            config=hyperparam)
for epoch in range(len(history.history["loss"])):
    wandb.log({
        "train_loss": history.history["loss"][epoch],
        "train_accuracy": history.history["mean_px_acc"][epoch],
        "train_auc": history.history["auc"][epoch],
        "train_precision": history.history["precision"][epoch],
        "train_recall": history.history["recall"][epoch],
        "val_loss": history.history["val_loss"][epoch],
        "val_accuracy": history.history["val_mean_px_acc"][epoch],
        "val_auc": history.history["val_auc"][epoch],
        "val_precision": history.history["val_precision"][epoch],
        "val_recall": history.history["val_recall"][epoch],
    })
wandb.log_artifact(os.path.join(path_model, f"{file_model_name}.h5"))
wandb.finish()

train_accuracy,▁▃▅▆▇▇████
train_auc,▁▅▆▇▇▇████
train_loss,█▄▄▃▃▂▂▁▁▁
train_precision,▁▅▆▆▇▇████
train_recall,▁▃▄▄▅▇▇███
val_accuracy,▁█████████
val_auc,▆█▃▃▂▂▁▁▁▁
val_loss,▂▁▃▃▄▅▆▇█▇
val_precision,▁▁██▄▄▃▃▃▃
val_recall,▁▁▁▂▆█████
train_accuracy,0.96723


In [11]:
api.upload_file(
    repo_id=repo.repo_id,
    path_or_fileobj=os.path.join(path_model, f"{file_model_name}.h5"),
    path_in_repo=f"model/{file_model_name}.h5",
    repo_type=repo.repo_type,
    commit_message=f"aug - {hf_commit_message}",
)

unet_model_aug.h5: 100%|██████████| 1.57M/1.57M [00:01<00:00, 971kB/s] 


CommitInfo(commit_url='https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation/commit/795935712506c87bcce6caaf200a8e052cfb4e3f', commit_message='aug - implement weight training', commit_description='', oid='795935712506c87bcce6caaf200a8e052cfb4e3f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation', endpoint='https://huggingface.co', repo_type='model', repo_id='bugi-sulistiyo/CAD-Glaucoma_Segmentation'), pr_revision=None, pr_num=None)